In [ ]:
import requests, urllib3
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime, timedelta
import csv
import pandas as pd
import os
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np

In [ ]:
# Spotify API credentials
client_id = 'dfb4f42769e24d8a9870f81c6b660989'
client_secret = "PRIVATE INFO"

In [ ]:
session = requests.Session()

retry = urllib3.Retry(
    total=0,
    connect=None,
    read=0,
    allowed_methods=frozenset(['GET', 'POST', 'PUT', 'DELETE']),
    status=0,
    backoff_factor=0.3,
    status_forcelist=(429, 500, 502, 503, 504),
    respect_retry_after_header=False  # <---
)

adapter = requests.adapters.HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
)

In [ ]:
# Playlist ID - Top 50 Polska
playlist_id = '37i9dQZEVXbN6itCcaL3Tt'
# '37i9dQZF1DXcBWIGoYBM5M' - Top Global


In [ ]:
# Określ nazwę katalogu głównego projektu
project_directory_name = 'BigDataProject'

# Pobierz ścieżkę do katalogu, w którym znajduje się bieżący skrypt (SpotipyScript.ipynb)
current_directory = os.path.dirname(os.path.abspath('__file__')).replace('\\','/')

# Przejście w górę po drzewie katalogów, aż dojdziesz do katalogu projektu
while os.path.basename(current_directory) != project_directory_name:
    current_directory = os.path.dirname(current_directory).replace('\\','/')

project_directory = current_directory

In [ ]:
def save_to_csv(track_info, csv_file_path):
    # Sprawdź czy ścieżka kończy się na .csv
    if not csv_file_path.lower().endswith('.csv'):
        # Ustaw typ pliku na csv
        csv_file_path = csv_file_path + ".csv"

    # Pobierz katalog z pełnej ścieżki
    directory = os.path.dirname(csv_file_path)

    # Sprawdź czy katalog istnieje, a jeśli nie, utwórz go (rekurencyjnie)
    os.makedirs(directory, exist_ok=True)

    # Jeśli dane wejściowe to DataFrame, użyj wbudowanej funkcji to_csv()
    if isinstance(track_info, pd.DataFrame):
        track_info.to_csv(csv_file_path, mode='w' if not os.path.exists(csv_file_path) else 'a', index=False)
    else:
        # Jeśli dane wejściowe to słownik, obsłuż ręcznie zapis do pliku CSV
        fieldnames = list(track_info.keys())

        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=';')

            # Sprawdź, czy plik CSV jest pusty, a jeśli tak, napisz nagłówki
            if file.tell() == 0:
                writer.writeheader()

            # Zapisz informacje do pliku CSV
            if isinstance(track_info[fieldnames[0]], list):
                # Jeśli wartości są listami, przekształć listy wierszy
                writer.writerows({key: value[i] for key, value in track_info.items()} for i in range(len(track_info[fieldnames[0]])))
            else:
                # Jeśli wartości nie są listami, zapisz jedną linię
                writer.writerow(track_info)


In [ ]:
def load_existing_tracks(csv_file_path,sep=';',parse_dates=['date_added']):
    # Wczytaj istniejące informacje o utworach z pliku CSV
    try:
        existing_tracks = pd.read_csv(csv_file_path, sep=sep, parse_dates=parse_dates)
        
        return existing_tracks
    except FileNotFoundError:
        return None

In [ ]:
def check_for_new_rank(csv_file_path, top_tracks):
    # Sprawdź datę dodania pierwszego utworu z playlisty
    first_track_date_added = datetime.strptime(top_tracks['items'][0]['added_at'], '%Y-%m-%dT%H:%M:%SZ') if top_tracks['items'] else None

    # Jeśli csv_file_path to ścieżka do katalogu, nadaj nazwę nowemu pliku
    if os.path.isdir(csv_file_path):
        csv_file_path = os.path.join(csv_file_path, f"{first_track_date_added.date()}.csv").replace('\\', '/')
    else:
        # Jeśli katalog nie istnieje, utwórz go
        os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
        csv_file_path = csv_file_path.replace('\\', '/')

    # Sprawdź, czy plik CSV istnieje
    if os.path.exists(csv_file_path):
        existing_tracks = load_existing_tracks(csv_file_path)
        
        # Sprawdź, czy plik istnieje, jest pusty, lub różni się datą od pierwszego utworu
        if existing_tracks is None or existing_tracks.empty or existing_tracks['date_added'].iloc[-1].date() != first_track_date_added.date():
            return True
    else:
        # Plik nie istnieje, więc na pewno istnieje nowy ranking
        return True

    # Istnieje nowy ranking
    return False

In [ ]:
def retrive_track_info(playlist_id, csv_file_path='data/'): 
    try:
        # Pobierz top 50 utworów z playlisty
        top_tracks = sp.playlist_tracks(playlist_id, limit=50)

        # Sprawdź datę dodania pierwszego utworu z playlisty
        first_track_date_added = datetime.strptime(top_tracks['items'][0]['added_at'], '%Y-%m-%dT%H:%M:%SZ') if top_tracks['items'] else None

        # Stwórz zmienną informującą o istnieniu nowych danych
        new_rank_exists = check_for_new_rank(csv_file_path, top_tracks)
        
        if not new_rank_exists:
            return print("Brak nowych utworów do dodania.")     
        else:
            # Jeśli csv_file_path to ścieżka do katalogu, nadaj nazwę nowemu pliku
            if os.path.isdir(csv_file_path):
                csv_file_path = os.path.join(csv_file_path, f"{first_track_date_added.date()}.csv").replace('\\', '/')
            else:
                # Jeśli katalog nie istnieje, utwórz go
                os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
                csv_file_path = csv_file_path.replace('\\', '/')

            # Przygotuj słownik informacji o utworach do zapisu
            track_info_dict = {
                'track_id': [track['track']['id'] for track in top_tracks['items']],
                'rank': list(range(1, len(top_tracks['items']) + 1)),
                'track_name': [track['track']['name'] for track in top_tracks['items']],
                'artist': [track['track']['artists'][0]['name'] for track in top_tracks['items']],
                'date_added': [datetime.strptime(track['added_at'], '%Y-%m-%dT%H:%M:%SZ') for track in top_tracks['items']],
                'popularity': [track['track']['popularity'] for track in top_tracks['items']]
            }
            # Zapisz informacje o utworach do pliku CSV
            save_to_csv(track_info_dict, csv_file_path)
            
            # Wyświetl informacje o utworach
            for i in range(len(track_info_dict['rank'])):
                print(f"{track_info_dict['rank'][i]}. {track_info_dict['track_name'][i]} by {track_info_dict['artist'][i]} (Popularity: {track_info_dict['popularity'][i]})")

    except spotipy.SpotifyException as e:
        print(f"Spotify API error: {e}")
    except Exception as e:
        print(f"Error: {e}")

    return

In [ ]:
def seperate_tracks_by_date(source_file_path, result_file_path=None): # IN CASE OF EMERGENCY - na wypadek usunięcia plików z danymi odrębnymi dla danej daty
    # Funkcja rozdziela zbiorczy plik csv z top utworami, na pliki z top utworami dla danego dnia

    # Wczytaj plik CSV, ze sprawdzeniem typu pliku
    if not source_file_path.lower().endswith('.csv'):
        return "Plik nie jest plikiem csv"
    else:
        df = pd.read_csv(source_file_path, parse_dates=['date_added'], sep=';')

    # Zdefiniuj ścieżkę plików wynikowych
    if not result_file_path:
        result_file_path = os.path.dirname(source_file_path) +"/"

    # Grupuj DataFrame według kolumny 'date_added'
    grouped_by_date = df.groupby(df['date_added'].dt.date)

    # Dla każdej grupy utwórz osobny plik CSV
    for date, group in grouped_by_date:
        # Utwórz nazwę pliku na podstawie daty
        file_name = result_file_path + f"{date}.csv"
        
        # Zapisz grupę do pliku CSV
        group.to_csv(file_name, index=False, sep=';')

    print("Pliki CSV zostały utworzone dla każdego unikalnego dnia.")


In [ ]:
# Dopasuj ID utworów na podstawie wiersza z ramki - początkowo ramki dzienne nie zawierały track_id
def get_track_id(row):
    track_name = row['track_name']
    artist = row['artist']
    track_id = row.get('track_id')  # Sprawdź, czy track_id już istnieje

    # Jeśli track_id już istnieje, nie wykonuj wyszukiwania
    if track_id:
        return track_id

    # Wyszukiwanie utworu
    results = sp.search(q=f"track:{track_name} artist:{artist}", type='track', limit=1)

    # Sprawdź, czy znaleziono utwór
    if results['tracks']['items']:
        found_track = results['tracks']['items'][0]
        track_id = found_track['id']
        return track_id
    else:
        return None

In [ ]:
def add_track_id_to_csv(csv_file_path):
    # Wczytaj ramkę danych CSV
    df = load_existing_tracks(csv_file_path)

    # Sprawdź, czy wczytano prawidłową ramkę danych
    if df is not None:
        # Zastosuj funkcję do każdego wiersza w ramce danych
        df['track_id'] = df.apply(get_track_id, axis=1)

        # Przesuń kolumnę 'id' na pierwszą pozycję
        cols = ['track_id'] + [col for col in df if col != 'track_id']
        df = df[cols]

        # Zapisz nową ramkę danych pod starą ścieżką
        df.to_csv(csv_file_path, sep=';', index=False)
        print('Zmodyfikowano ramkę')
    else:
        print("Wczytywanie pliku CSV nie powiodło się.")

In [ ]:
def apply_to_date_files(start_date,end_date,directory_path):
    # Iterate through files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        try:
            # Check if the file is a CSV file and its name represents a date within the specified range
            if filename.endswith('.csv') and start_date <= filename <= end_date:
                add_track_id_to_csv(file_path)
                print("Done")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

In [ ]:
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

In [ ]:
def convert_csv_to_parquet(csv_file_path, output_dir, file_name, sep=';', parse_dates=['date_added']): 
    ### robocze zapisywanie plików do .parquet,  aby wczytywać je z hdfs przy pracy z pySpark ###

    # Wczytaj istniejące informacje o utworach z funkcji
    existing_tracks = load_existing_tracks(csv_file_path, sep, parse_dates)

    if existing_tracks is not None:
        if file_name is None:
            # Utwórz nazwę pliku wynikowego w formacie Parquet
            output_file_name = 'songs_' + csv_file_path.split('/')[-1].split('.')[0] + '.parquet'
            output_file_path = os.path.join(output_dir, output_file_name).replace('\\','/')
            
            # Sprawdź, czy plik Parquet już istnieje
            if os.path.exists(output_file_path):
                return print("Taki plik już istnieje")
        else:
            # Utwórz nazwę pliku wynikowego w formacie Parquet
            output_file_path = os.path.join(output_dir, file_name + '.parquet').replace('\\','/')

            # Sprawdź, czy plik Parquet już istnieje
            if os.path.exists(output_file_path):
                os.remove(output_file_path)  # Usuń istniejący plik
                print('Znaleziono istniejący plik - zostanie on podmieniony')

        # Zapisz dane w formacie Parquet
        table = pa.Table.from_pandas(existing_tracks)
        pq.write_table(table, output_file_path)

        print(f"Plik Parquet został utworzony: {output_file_path}")
    else:
        print("Nie można znaleźć pliku CSV.")


In [ ]:
def convert_data_csv_to_parquet(input_directory, output_directory, file_name, sep=';', parse_dates=['date_added']): # wersja dla katalogu, który zawiera m.in. ramki daily
    ### robocze zapisywanie plików do .parquet,  aby wczytywać je z hdfs przy pracy z pySpark ###

    # Sprawdź, czy katalog istnieje
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Lista plików w katalogu wejściowym
    files = os.listdir(input_directory)

    for file in files:
        # Sprawdź, czy plik ma odpowiedni format nazwy
        if file.endswith('.csv') and is_valid_date(file.split('.')[0]):
            # Utwórz pełną ścieżkę do pliku CSV
            csv_path = os.path.join(input_directory, file)
            convert_csv_to_parquet(csv_path, output_directory,file_name,sep= sep, parse_dates= parse_dates)

In [ ]:
def get_audio_features(track_ids): # poprawianie ramki statycznej 
    
    # Sprawdzenie czy track_ids jest listą lub stringiem
    if not (type(track_ids) is list) and not (type(track_ids) is str):
        return print("Podaj listę lub string z track_id")

    audio_features_list = sp.audio_features(tracks=track_ids)

    # Przygotowanie danych do zapisu
    audio_feat= {
        'track_id': [audio_features['id'] for audio_features in audio_features_list],
        'danceability': [audio_features['danceability'] for audio_features in audio_features_list],
        'energy': [audio_features['energy'] for audio_features in audio_features_list],
        'key': [audio_features['key'] for audio_features in audio_features_list],
        'loudness': [audio_features['loudness'] for audio_features in audio_features_list],
        'mode': [audio_features['mode'] for audio_features in audio_features_list],
        'speechiness': [audio_features['speechiness'] for audio_features in audio_features_list],
        'acousticness': [audio_features['acousticness'] for audio_features in audio_features_list],
        'instrumentalness': [audio_features['instrumentalness'] for audio_features in audio_features_list],
        'liveness': [audio_features['liveness'] for audio_features in audio_features_list],
        'valence': [audio_features['valence'] for audio_features in audio_features_list],
        'tempo': [audio_features['tempo'] for audio_features in audio_features_list],
        'duration_ms': [audio_features['duration_ms'] for audio_features in audio_features_list]
    }
    return audio_feat

In [ ]:
def get_track_info(track_ids): # poprawianie ramki statycznej 
    """
    Get track information for a list of track IDs.

    Parameters:
    - track_ids (list or str): List of track IDs or a single track ID as a string.

    Returns:
    - track_info (dict): Dictionary containing track information.
    """
    # Sprawdzenie czy track_ids jest listą lub stringiem
    if not (isinstance(track_ids, list) or isinstance(track_ids, str)):
        raise ValueError("Podaj listę lub string z track_id")

    # Jeśli track_ids to string, zamień go na listę zawierającą ten string
    if isinstance(track_ids, str):
        track_ids = [track_ids]

    # Pobierz informacje o utworach
    tracks_info = sp.tracks(tracks=track_ids)['tracks']

    # Przygotuj dane do zapisu
    track_info = {
        'track_id': [track['id'] for track in tracks_info],
        'track_name': [track['name'] for track in tracks_info],
        'track_artist': [track['artists'][0]['name'] for track in tracks_info],
        'track_album_release_date': [track['album']['release_date'] for track in tracks_info],
        'track_album_id': [track['album']['id'] for track in tracks_info],
        'artist_id': [track['artists'][0]['id'] for track in tracks_info],
        'track_album_name': [track['album']['name'] for track in tracks_info]
    }

    return track_info

In [ ]:
def get_artist_genres(artist_ids): # poprawianie ramki statycznej 
    """
    Get genres for a list of artist IDs.

    Parameters:
    - artist_ids (list or str): List of artist IDs or a single artist ID as a string.

    Returns:
    - artist_genres (dict): Dictionary containing artist IDs and their associated genres.
    """
    # Sprawdzenie czy artist_ids jest listą lub stringiem
    if not (isinstance(artist_ids, list) or isinstance(artist_ids, str)):
        raise ValueError("Podaj listę lub string z artist_id")

    # Jeśli artist_ids to string, zamień go na listę zawierającą ten string
    if isinstance(artist_ids, str):
        artist_ids = [artist_ids]

    # Pobierz informacje o artystach
    artists_info = sp.artists(artist_ids)['artists']

    # Przygotuj dane do zapisu
    artist_genres = {
        'artist_id': [artist['id'] for artist in artists_info],
        'artist_genres': [artist['genres'] if artist['genres'] else ['unknown'] for artist in artists_info]
    }

    return artist_genres

In [ ]:
def get_full_data(track_ids): # poprawianie ramki statycznej 
    # Pobierz informacje o utworach
    track_info = get_track_info(track_ids)
    track_info_df = pd.DataFrame(track_info)

    # Pobierz cechy audio utworów
    audio_features = get_audio_features(track_ids)
    audio_features_df = pd.DataFrame(audio_features)

    # Połącz ramki danych na podstawie 'track_id'
    full_data = pd.merge(track_info_df, audio_features_df, on='track_id', how='inner')

    # Pobierz unikalne artist_ids
    unique_artist_ids = full_data['artist_id'].unique()

    # Pobierz informacje o gatunkach artystów
    artist_genres = get_artist_genres(list(unique_artist_ids))
    artist_genres_df = pd.DataFrame(artist_genres)


    # Połącz ramki danych na podstawie 'artist_id'
    full_data = pd.merge(full_data, artist_genres_df, on='artist_id', how='left')
    print(f" Dodano: {full_data.shape[0]} nowych utworów")

    return full_data

In [ ]:
def update_song_base(csv_file_path, track_ids): # poprawianie ramki statycznej - dodawanie utworów z Top 50, których cech nie było oryginalnie w ramce
    # Wczytaj istniejące dane z pliku CSV
    if os.path.exists(csv_file_path):
        existing_data = pd.read_csv(csv_file_path)
        existing_track_ids = set(existing_data['track_id'].unique())
    else:
        # Jeśli plik nie istnieje, utwórz pustą ramkę danych
        existing_data = pd.DataFrame()
        existing_track_ids = set()

    # Sprawdź, czy track_ids to lista, jeśli nie, zamień go na listę
    if isinstance(track_ids, str):
        track_ids = [track_ids]

    # Wybierz unikalne track_id (bez duplikatów)
    new_track_ids = set(track_ids) - existing_track_ids

    # Jeśli są nowe track_id, pobierz ich pełne dane
    if new_track_ids:
        new_data = get_full_data(list(new_track_ids))

        # Dołącz nowe dane do istniejących danych
        updated_data = pd.concat([existing_data, new_data], ignore_index=True)

        # Zapisz zaktualizowane dane do pliku CSV
        updated_data.to_csv(csv_file_path, index=False)
        print("Dane zostały zaktualizowane.")
    else:
        print("Brak nowych danych do dodania.")

In [ ]:
def chunks(list, chunk_size): # aby uniknąć przeciążenia API, gdy trzeba pobrać informacje o wielu utworach
    return [list[i:i + chunk_size] for i in range(0, len(list), chunk_size)]

In [ ]:
def df_for_artist_info(chunk, csv_file_path): # poprawianie ramki statycznej 
    """
    Update the dataframe with track information for a given chunk of track IDs.

    Parameters:
    - chunk (list): List of track IDs.
    - csv_file_path (str): Path to the CSV file.

    Returns:
    - updated_dataframe (pd.DataFrame): Updated DataFrame with added track information.
    """
    # Sprawdź czy plik istnieje
    if os.path.exists(csv_file_path):
        # Wczytaj istniejący plik CSV do ramki danych
        dataframe = pd.read_csv(csv_file_path)
    else:
        # Jeśli plik nie istnieje, stwórz pustą ramkę danych
        dataframe = pd.DataFrame()

    # Sprawdź, czy track_id z aktualnego chunka już istnieje w ramce danych
    existing_track_ids = set(dataframe['track_id'].unique())
    new_track_ids = set(chunk) - existing_track_ids

    if not new_track_ids:
        print("Wszystkie track_id z tego chunka już istnieją w ramce danych. Pomijam.")
        return dataframe

    # Pobierz informacje o utworach
    track_info = get_track_info(list(new_track_ids))

    # Pobierz artist_id z track_info
    artist_ids = track_info['artist_id']

    # Pobierz informacje o artystach
    artist_genres = get_artist_genres(artist_ids)

    # Stwórz słownik z informacjami o trackach, artystach i gatunkach
    update_dict = {
        'track_id': track_info['track_id'],
        'artist_id': track_info['artist_id'],
        'artist_genres': artist_genres['artist_genres']
    }

    # Jeśli dataframe jest pusty lub kolumny w update_dict zgadzają się z istniejącymi kolumnami,
    # dołącz do niego nowe informacje
    if dataframe.empty or set(update_dict.keys()).issubset(set(dataframe.columns)):
        updated_dataframe = pd.concat([dataframe, pd.DataFrame(update_dict)], ignore_index=True)
    else:
        # Dodaj odpowiednie obsługi błędów lub dostosuj kolumny, jeśli to konieczne
        print("Błąd: Kolumny w update_dict nie zgadzają się z istniejącymi kolumnami w dataframe.")
        updated_dataframe = dataframe

    # Zapisz zaktualizowaną ramkę do pliku CSV
    updated_dataframe.to_csv(csv_file_path, index=False)

    return updated_dataframe


In [ ]:
def process_chunks(to_chunk_param,chunk_size=50, processing_function=None,  **kwargs):
    track_id_chunks = chunks(kwargs.get(to_chunk_param, []), chunk_size)
    total_chunks = len(track_id_chunks)

    results_list = []

    for i, chunk in enumerate(track_id_chunks, start=1):
        print(f"Przetwarzanie chunka {i} z {total_chunks}")
        if processing_function is not None:
            kwargs_copy = kwargs.copy()
            kwargs_copy[to_chunk_param] = chunk
            results_chunk = processing_function(**kwargs_copy)
            results_list.append(results_chunk)
        else:
            print("Brak funkcji przetwarzającej.")

    return results_list


In [ ]:
def get_track_popularity(track_ids): # poprawianie ramki statycznej 
    """
    Get track information for a list of track IDs.

    Parameters:
    - track_ids (list, str, or DataFrame): List of track IDs, a single track ID as a string, or a DataFrame with a 'track_id' column.

    Returns:
    - track_info_df (DataFrame): DataFrame containing track information, including 'track_id' and 'popularity'.
    """
    if isinstance(track_ids, pd.DataFrame):
        # Handle DataFrame case
        print("Processing DataFrame...")
        # Utwórz nową kolumnę w DataFrame: 'popularity'
        track_ids = track_ids.copy()
        track_ids['popularity'] = np.nan

        # Znajdź wiersze, które zawierają NaN w kolumnie 'track_id'
        nan_rows = track_ids['track_id'].isna()
        num_nan_rows = nan_rows.sum()

        if num_nan_rows > 0:
            print(f"{num_nan_rows} utworów nie posiada track id.")

        # Dla wierszy z NaN w kolumnie 'track_id', przypisz NaN do nowej kolumny
        track_ids.loc[nan_rows, 'popularity'] = np.nan

        # Dla wierszy bez NaN w kolumnie 'track_id', pobierz informacje o popularności z Spotify
        non_nan_rows = ~nan_rows
        if non_nan_rows.any():
            non_nan_track_ids = track_ids.loc[non_nan_rows, 'track_id'].tolist()
            tracks_info = sp.tracks(tracks=non_nan_track_ids)['tracks']
            track_ids.loc[non_nan_rows, 'popularity'] = [track['popularity'] for track in tracks_info]

    else:
        # Handle list or string case
        # Sprawdzenie czy track_ids jest listą lub stringiem
        if not (isinstance(track_ids, list) or isinstance(track_ids, str)):
            raise ValueError("Podaj listę, string lub DataFrame z track_id")

        # Jeśli track_ids to string, zamień go na listę zawierającą ten string
        if isinstance(track_ids, str):
            track_ids = [track_ids]

        # Pobierz informacje o utworach
        tracks_info = sp.tracks(tracks=track_ids)['tracks']

        # Przygotuj dane do zapisu
        track_info = {
            'track_id': [track['id'] for track in tracks_info],
            'popularity': [track['popularity'] for track in tracks_info]
        }

        # Zamień słownik na ramkę danych
        track_ids = pd.DataFrame(track_info)

    return track_ids

In [ ]:
def save_yesterday_rank(excel_file_path): # na wypadek niepobrania danych z danego dnia można je wciąż uzyskać dnia następnego
    ##ZAPISAIE DANYCH Z DNIA POPRZEDNIEGO NA PODSTAWIE https://kworb.net/spotify/country/pl_daily.html

    # Zapisanie xlsx jako csv
    print('-------------------------------------- Zapisz w formacie csv...')
    # Wczytaj plik Excel
    df = pd.read_excel(excel_file_path)

    # Zapisz do pliku CSV
    csv_file_path = excel_file_path.replace('.xlsx','.csv')
    df.to_csv(csv_file_path, index=False, sep=';')
    # display(df)

    # Rozdzielenie track_name i artist
    print('\n-------------------------------------- Rozdzielenie track_name i artist...')
    # Funkcja do rozdzielenia kolumny 'Artist and Title'
    def split_artist_title(row):
        parts = row['Artist and Title'].split('-', 1)
        artist = parts[0].strip()
        track_name = parts[1].split('(w/', 1)[0].strip() if len(parts) > 1 else None
        return pd.Series([artist, track_name], index=['artist', 'track_name'])

    # Stosowanie funkcji do ramki danych
    df[['artist', 'track_name']] = df.apply(split_artist_title, axis=1)

    df.drop(columns='Artist and Title',axis=1, inplace=True)
    # Zapisz do pliku CSV
    # df.to_csv(csv_file_path, index=False, sep=';')
    # display(df)

    # Dodanie track_id na podstawie spotify_songs
    print('\n-------------------------------------- Dodanie track_id na podstawie spotify_songs...')
    df2 = load_existing_tracks(project_directory +'/src/data/spotify_songs.csv', sep=',',parse_dates= False)
    # Usuń duplikaty z df2
    df2 = df2.drop_duplicates(subset=['track_artist', 'track_name'])

    # Łączenie ramek danych na podstawie 'artist' i 'track_name'
    df = pd.merge(df, df2[['track_artist','track_name', 'track_id']], how='left', left_on=['artist', 'track_name'], right_on=['track_artist', 'track_name'])

    # Usuń dodatkowe kolumny z ramki df2
    df.drop(['track_artist'], axis=1, inplace=True)

    # Wyświetl wynikową ramkę danych
    #Zapisz do pliku CSV
    # df.to_csv(csv_file_path, index=False, sep=';')
    # display(df)

    #  Dodanie popularity
    print('\n-------------------------------------- Dodanie popularity...')
    # Pobierz track_id z ramki danych
    track_ids = df
    track_info_list = process_chunks( # Wywołanie w pakietach aby uniknąć przeciążenia API
        to_chunk_param='track_ids',
        chunk_size=10,
        processing_function=get_track_popularity,
        track_ids = track_ids
    )

    # Połącz DataFrame'y z listy
    df = pd.concat(track_info_list, ignore_index=True)

    # Dodanie daty
    print('\n-------------------------------------- Dodanie daty...')

    # Ścieżka do pliku top_tracks
    df3_file_path = project_directory + '/src/data/top_tracks.csv'
    # Wczytaj ramki danych
    df3 = load_existing_tracks(df3_file_path)

    # Pobierz nazwę pliku bez ścieżki i rozszerzenia
    file_name = csv_file_path.split("/")[-1].split(".")[0]

    # Oblicz datę na podstawie nazwy pliku i daty z df3
    day = (datetime.strptime(file_name, '%Y-%m-%d') - timedelta(days=7)).date()

    # Zamień kolumnę 'date_added' na typ datetime
    date_added = df3[df3['date_added'].dt.strftime('%Y-%m-%d') == str(day)]['date_added'].iloc[0] + timedelta(days=7)

    # Dodaj kolumnę "date_added" do ramki df
    df['date_added'] = date_added

    print('\n-------------------------------------- Poprawienie kolejności kolumn...')
    # Załóżmy, że masz aktualną kolejność kolumn
    current_columns = df.columns.tolist()

    # Nowa kolejność kolumn
    new_columns_order = ['track_id', 'rank', 'track_name', 'artist', 'date_added', 'popularity']

    # Upewnij się, że wszystkie kolumny są obecne w nowej kolejności
    assert set(current_columns) == set(new_columns_order), "Nowa kolejność kolumn musi zawierać wszystkie istniejące kolumny."

    # Ustaw nową kolejność kolumn w ramce danych
    df = df[new_columns_order]

    # Zapisz do pliku CSV
    df.to_csv(csv_file_path, index=False, sep=';')
    
    return print(f"Zapisano plik: {csv_file_path}. Zwróć uwagę na brakujące track_id i popularity")

In [ ]:
print('----------- Pobierz nowe rankingi --------------------------------------')
# Pobieranie utworów do oddzielnej ramki dla danego dnia
retrive_track_info(playlist_id,project_directory + '/src/data/csv/')

# Pobieranie utworów do zbiorczej ramki
retrive_track_info(playlist_id, project_directory + '/src/data/csv/top_tracks.csv')


# Uzyskaj dzisiejszą datę
todays_date = str(datetime.now().date())

# Załaduj dzisiejszy ranking i ścieżkę do bazy
todays_tracks_path = project_directory + '/src/data/csv/' + todays_date +'.csv'
todays_tracks = load_existing_tracks(todays_tracks_path)
base_path = project_directory + '/src/data/csv/spotify_songs.csv'

print('\n----------- Uzupełnij bazę piosenek --------------------------------------')
# Uzupełnij bazę piosenek o nowe z danego dnia i ich atrybuty - baza bez duplikatów

process_chunks( # Wywołanie w pakietach aby uniknąć przeciążenia API
    to_chunk_param = 'track_ids', 
    chunk_size=50, 
    processing_function=update_song_base,
    track_ids = list(set(todays_tracks['track_id'])),
    csv_file_path = base_path)

print('\n----------- Zapisz w formacie parquet --------------------------------------')

# Zamień format danych (bazy piosenek i pliku z dzisiejszym rankingiem) na .parquet i zapisz w odpowiednim katalogu
convert_csv_to_parquet(todays_tracks_path,project_directory+'/src/data/parquet/',file_name=None) # dzisiejszy ranking
convert_csv_to_parquet(base_path,os.path.dirname(base_path),file_name='spotify_songs',sep=',',parse_dates=False) # baza

# # Zamień wszystkie pliki danych z datą w nazwie na .parquet i zapisz w specjalnym katalogu
# convert_data_csv_to_parquet(project_directory+'/src/data/',project_directory+'/src/data/parquet/',file_name=None)